In [ ]:
# https://github.com/awslabs/autogluon
# AutoGluon is a SOTA AutoML framework
# This kernel showcases AutoGluon with RAPIDS GPU acceleration

!pip install "autogluon.tabular[all]==0.1.1b20210312"

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.models.knn.knn_rapids_model import KNNRapidsModel
from autogluon.tabular.models.lr.lr_rapids_model import LinearRapidsModel
train_data = TabularDataset('../input/otto-group-product-classification-challenge/train.csv')
test_data = TabularDataset('../input/otto-group-product-classification-challenge/test.csv')

label = 'target'

In [ ]:
# Fit end-to-end with raw data in under 1 hour, with one line of code
predictor = TabularPredictor(
    label=label,
    eval_metric='log_loss',
    learner_kwargs={'ignored_columns': ['id']}
).fit(
    train_data,
    presets='best_quality',
    hyperparameters={
        KNNRapidsModel: {},
        LinearRapidsModel: {},
        'RF': {},
        'XGB': {'ag_args_fit': {'num_gpus': 1}},
        'CAT': {'ag_args_fit': {'num_gpus': 1}},
        'GBM': [{}, {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, 'GBMLarge'],
        'NN': {'ag_args_fit': {'num_gpus': 1}},
        'FASTAI': {'ag_args_fit': {'num_gpus': 1}},
    },
)

In [ ]:
import pandas as pd
submission = test_data[['id']]
test_pred_proba = predictor.predict_proba(test_data)
submission = pd.concat([submission, test_pred_proba], axis=1)
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
predictor.leaderboard(silent=True)